In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass 
!google-drive-ocamlfuse -headless -id={crgoeds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

gpg: keybox '/tmp/tmpjviojmip/pubring.gpg' created
gpg: /tmp/tmpjviojmip/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=%7Bcrgoeds.client_id%7D&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
import numpy as np
home_dir ='./drive/ml_dir/test_CST/'
# 4/VgB6mgJJuGyXfUOPc8TGiwGVhB3u2BAPLtXHy6ztCGE8aEcJoPN3eug

In [4]:
# print(y_train[3])

# y_fixed = np.zeros((y_test.shape[0],8))
# for i in range(y_test.shape[0]):
#     label = int(y_test[i])
#     if(label ==8):
#         continue
#     y_fixed[i, label] = 1

# print(y_fixed[0:5])
# np.save(home_dir + 'data/y_test.npy', y_fixed)
! ls ./drive/ml_dir/test_CST/data

mbe_3sec_random_default .npy	   mfcc_default_15.npy
mbe_3sec_random_default _test.npy  mfcc_default_15_test.npy
mfcc3sec_random_15.npy		   mfcc_random_15.npy
mfcc3sec_random_15_test.npy	   mfcc_random_15_test.npy
mfcc3sec_random_20.npy		   y.npy
mfcc3sec_random_20_test.npy	   y_test.npy


In [4]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Activation, LSTM, Dropout
from keras.metrics import categorical_accuracy

x_train = np.load(home_dir + 'data/mbe_3sec_random_default .npy')
y_train = np.load(home_dir + 'data/y.npy')

x_test = np.load(home_dir + 'data/mbe_3sec_random_default_test3.npy')
y_test = np.load(home_dir + 'data/y_test.npy')

x_unknown = np.load(home_dir + 'data/x_unknown.npy')


model = Sequential()
model.add(LSTM(512, input_shape = (x_train.shape[1:]), return_sequences = True))
model.add(Dropout(0.7))
model.add(LSTM(256, return_sequences = False))


model.add(Dense(8))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[categorical_accuracy])


Using TensorFlow backend.


In [46]:
print(history.val_losses)

[0.505443307464888]


In [0]:
import matplotlib.pyplot as plt
import keras
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
    def on_epoch_end(self, epoch, logs={}):
        self.val_losses.append(logs.get('val_loss'))
 

In [11]:

model = Sequential()

model.add(LSTM(256, input_shape = (x_train.shape[1:]), return_sequences = False))


model.add(Dense(8))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[categorical_accuracy])
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01,  patience=2, verbose=0, mode='auto')

history = LossHistory()
model.fit(x_train, y_train, batch_size=1500, validation_split=0.1, epochs = 100, callbacks=[])

Train on 10176 samples, validate on 1131 samples
Epoch 1/100
10176/10176 [==============================] - 13s 1ms/step - loss: 1.7717 - categorical_accuracy: 0.3620 - val_loss: 1.4663 - val_categorical_accuracy: 0.5181
Epoch 2/100
10176/10176 [==============================] - 11s 1ms/step - loss: 1.3578 - categorical_accuracy: 0.5570 - val_loss: 1.2412 - val_categorical_accuracy: 0.6127
Epoch 3/100
10176/10176 [==============================] - 11s 1ms/step - loss: 1.1608 - categorical_accuracy: 0.6307 - val_loss: 1.1326 - val_categorical_accuracy: 0.6198
Epoch 4/100
10176/10176 [==============================] - 11s 1ms/step - loss: 1.0100 - categorical_accuracy: 0.6720 - val_loss: 0.9925 - val_categorical_accuracy: 0.6508
Epoch 5/100
10176/10176 [==============================] - 11s 1ms/step - loss: 0.9712 - categorical_accuracy: 0.6775 - val_loss: 1.0036 - val_categorical_accuracy: 0.6366
Epoch 6/100
10176/10176 [==============================] - 11s 1ms/step - loss: 0.9130 - ca

KeyboardInterrupt: ignored

In [12]:
from sklearn.metrics import classification_report
y_test = np.load(home_dir + 'data/y_test.npy')
print(y_test.shape)
print( classification_report(np.argmax(y_test, axis=1), np.argmax(model.predict(x_test) , axis = 1)))

# np.save(home_dir + 'results/ cr.npy', classification_report(np.argmax(y_train, axis=1), np.argmax(model.predict(x_train) , axis = 1)))


(473, 8)
             precision    recall  f1-score   support

          0       0.70      0.93      0.80        41
          1       0.98      0.80      0.88        50
          2       0.44      0.86      0.58        49
          3       0.79      0.89      0.84        47
          4       0.45      0.95      0.61        59
          5       0.96      0.60      0.74        72
          6       0.98      0.31      0.47       141
          7       0.53      0.57      0.55        14

avg / total       0.80      0.66      0.65       473



In [13]:
parameters = [64, 128, 256 , 512]
dropouts = [0.5, 0.6 , 0.7, 0.8]
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
for param in parameters:
    for param2 in parameters:
        for drop in dropouts:

            model = Sequential()
            model.add(LSTM(param, input_shape = (x_train.shape[1:]), return_sequences = True))
            model.add(Dropout(drop))
            model.add(LSTM(param2, return_sequences = False))


            model.add(Dense(8))
            model.add(Activation('softmax'))
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[categorical_accuracy])

            history = LossHistory()
            model.fit(x_train, y_train, batch_size=128, validation_split=0.1, epochs = 25, callbacks=[history, earlyStopping])
            np.save(home_dir + 'results/{0}_{1}_{2}.npy'.format(str(param),str(param2),str(drop)), np.array([history.losses, history.val_losses]))
            np.save(home_dir + 'results/cr_{0}_{1}_{2}.npy'.format(str(param),str(param2),str(drop)), classification_report(np.argmax(y_test, axis=1), np.argmax(model.predict(x_test) , axis = 1)))



KeyboardInterrupt: ignored

In [0]:
model.save(home_dir + 'mymodel')

# Best Model is 512-256 LSTM + mbe

---

